
# Databricks para todos. Una Introducción a la creación de agentes en Databricks

## 1. ¿Qué es Databricks?


Databricks es una plataforma unificada de análisis y gestión de datos que integra todo el ciclo de vida de los datos: desde procesos ETL, BI, IA, Big Data, streaming, cómputo y desarrollo, hasta muchas otras funcionalidades. Permite construir, desplegar y mantener soluciones empresariales de datos y analítica a escala, integrándose con el almacenamiento y la seguridad en la nube. Databricks facilita el procesamiento, almacenamiento y análisis de datos, optimizando el rendimiento y simplificando la infraestructura para aportar valor en el mercado.



<img src="https://learn.microsoft.com/en-us/azure/databricks/_static/images/what-is/data_intelligence_engine.png" alt="drawing" width="400"/>


https://learn.microsoft.com/en-us/azure/databricks/_static/images/what-is/data_intelligence_engine.png


## 2. ¿Qué es un Agente?


Un agente es un sistema impulsado por IA que puede percibir, razonar y actuar de manera autónoma para lograr objetivos. Utiliza un LLM como “cerebro” para interpretar el contexto, decidir qué acciones tomar y ejecutar tareas, combinando modelos de lenguaje y herramientas para automatizar procesos que antes requerían intervención humana.


### 2.1 Agente vs un LLM

- Los Agentes toman decisiones los LLM generan respuestas



### 2.2 La arquitectura de un Agente

In [0]:
%%mermaid
flowchart TD
  User([Usuario])
  Agent([Agente])
  PromptTemplate[Plantilla de Prompt]
  LLM[LLM]
  Tools[Herramientas]
  Memory[Memoria]

  User -- "Prompt" --> Agent
  Agent -- "Respuesta" --> User
  PromptTemplate -- "Instrucciones / Comportamiento" --> Agent
  Agent -- "Planificación / Razonamiento / El cerebro" --> LLM
  Agent -- "Acciones" --> Tools
  Agent -- "Almacena/Recupera" --> Memory



### 2.3 Patrones de diseño de los Agentes


#### 2.3.1 El Patrón ReAct
el patrón ReAct (Reasoning + Acting) describe un proceso iterativo donde un LLM:

- Recibe un prompt inicial.
- Piensa o razona paso a paso sobre cómo responder o avanzar.
- Ejecuta una acción (que puede ser generar una pregunta adicional para recopilar más información o usar herramientas externas),
- Incorpora la nueva información obtenida.
- Y repite este ciclo hasta que considere tener suficiente contexto para dar una respuesta completa y acertada al prompt original.

<img src="https://react-lm.github.io/files/hotpotqa.png"
 alt="drawing" width="1000"/>

https://arxiv.org/pdf/2210.03629



#### 2.3.2 Planner Executor: Planificación Jerárquica y Ejecución
El patrón Planner Executor consiste en dividir tareas complejas en subtareas mediante un planificador (Planner) que genera una secuencia ordenada de pasos y un ejecutor (Executor) que materializa cada uno de estos pasos. Esta descomposición jerárquica facilita la paralelización, la gestión de errores y la reutilización de componentes. Es especialmente útil para la automatización de procesos empresariales y la orquestación de flujos de trabajo tanto en entornos cloud como on premises. El patrón permite un control estructurado y eficiente, optimizando la ejecución escalonada y monitoreada de cada sub-tarea.

#### 2.3.3 Agentes con Herramientas y Conectores
Este patrón se basa en potenciar las capacidades de los agentes de IA integrándolos con herramientas externas como motores de búsqueda, sistemas ERP, servicios en la nube o plataformas analíticas. Mediante conectores bien diseñados, los agentes pueden acceder a datos en tiempo real, ejecutar scripts automáticos o generar informes dinámicos en herramientas como Power BI. Este enfoque es esencial para integrar agentes inteligentes en soluciones de negocio complexas y personalizadas, permitiendo la conexión fluida con servicios cloud y aplicaciones de terceros para ampliar su funcionalidad.

#### 2.3.4 Autoverificación y Reflexión
El patrón de Autoverificación incorpora ciclos iterativos de reflexión donde el agente revisa críticamente sus propias decisiones y resultados, validando la coherencia, precisión y consistencia de sus acciones. Esto incluye mecanismos automáticos para detectar y corregir errores, lo que aumenta la fiabilidad del sistema en tareas críticas como la generación de código, análisis financiero y decisiones de ciberseguridad. La implementación de métricas de confianza y chequeos internos es fundamental para asegurar la robustez del agente en entornos productivos.

#### 2.3.5 Colaboración Multiagente
Este patrón describe la cooperación entre varios agentes especializados que trabajan como un equipo distribuido, comunicándose, negociando roles y complementando sus capacidades para resolver problemas complejos. La arquitectura multiagente permite integrar expertos en diversas áreas (datos, seguridad, negocio, automatización) para ofrecer soluciones más completas y escalables. Este enfoque es especialmente potente cuando se combina con servicios avanzados de inteligencia de negocio y dashboards que facilitan la supervisión y coordinación del sistema multiagente.


### 2.4 Herramientas para crear Agentes

<img src="/Workspace/Users/santiago.rincon554@gmail.com/data/Images/Captura de pantalla 2025-11-19 a la(s) 10.03.58 p.m..png"
 alt="drawing" width="1000"/>

 https://campus.datacamp.com/courses/introduction-to-ai-agents/foundations-of-ai-agents?ex=7

| Herramientas "Off-the-Shelf"                                         | Plataformas Low-Code/No-Code                                              | Frameworks de Agentes (Construcción)                          |
|----------------------------------------------------------------------|---------------------------------------------------------------------------|---------------------------------------------------------------|
| Enfocados en dominio o caso de uso específico (ej. código asistido por IA) | Requieren personalización pero sin control total                       | El caso de uso involucra sistemas propietarios                 |
| Solución madura y probada ya existe en el mercado                    | Los flujos de trabajo son moderadamente complejos y siguen patrones comunes | Manejo de datos sensibles                                     |
| Se busca minimizar el mantenimiento                                  | Usuarios de negocio pueden modificar el agente sin ingeniería           | El agente es clave para la ventaja competitiva                 |
|                                                                      | Necesidad de integrar rápido con sistemas existentes                    | No existe solución que cubra requisitos especializados         |
|                                                                      |                                                                           | Se necesita control total sobre el comportamiento y evolución del agente |



## 3. Creación de un Agente con LangChain y Databricks models


La siguiente demostración consiste en un proceso de extración y análisis de datos utilizando agentes.

Extraemos datos de kaggle y los almacenamos en databricks. El modelo, mediante herramientas, lee y realiza transformaciones de los datos como se le indica mediante un prompt y, al final, hace un pequeño analisis de la información.


### 3.1 Los datos

El dataset “Mexican Federal Government Salaries” contiene datos de salarios (en MXN) de empleados federales mexicanos

https://www.kaggle.com/datasets/ivansabik/mexican-federal-government-salaries

In [0]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = "Tokens/Kaggle"

!kaggle datasets download -d ivansabik/mexican-federal-government-salaries -p /data/kaggle-datasets --unzip

In [0]:
import pandas as pd

csv_path = "../data/kaggle-datasets/salaries.csv"

pd.read_csv(csv_path).head()

,entidadfederativa,sujetoobligado,nombre,denominacion,montoneto,cargo,area,montobruto,idInformacion,periodoreportainicio,periodoreportafin
0,Hidalgo,Jaltocán,Adolfo Hernandez Hernandez,Fontanero,4000.00,Fontanero,OBRAS PUBLICAS,4254.00,16311845,01/01/2018,30/06/2018
1,Ciudad de México,Secretaría de Salud,ARELY SAMANTA CLEOFAS VELASCO,"AUXILIAR DE ENFERMERIA ""A""",12177.86,"AUXILIAR DE ENFERMERIA ""A""",H.G. ENRIQUE CABRERA,16092.00,16480190,01/01/2018,31/03/2018
2,Ciudad de México,Secretaría de Seguridad Ciudadana,MELODY OLIMPIC GONZALEZ MONTES,POLICIA PRIMERO,11652.00,POLICIA PRIMERO,SUBSECRETARIA DE OPERACION POLICIAL,16030.00,17599078,01/01/2020,31/03/2020
3,Federación,Autoridad Educativa Federal en la Ciudad de Mé...,ANGEL ALLENDE PULIDO,APOYO Y ASISTENCIA A LA EDUCACION,10180.57,APOYO Y ASISTENCIA A LA EDUCACION,DIRECCIÓN GENERAL DE OPERACIONES DE SERVICIOS ...,2910.65,6514612,01/07/2018,31/12/2018
4,Aguascalientes,MUNICIPIO DE RINCÓN DE ROMOS,Yolanda Reyes Gonzalez,DIRECTOR,17004.40,DIRECTOR,ACCION CIVICA,6188.40,11927166,01/07/2019,31/12/2019



### 3.2 El cerebro

In [0]:
from databricks_langchain.chat_models import ChatDatabricks

llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct", #Gratis con limitaciones dentro de entornos Databricks
    max_tokens=1000, # Que tan grande debe ser la respuesta del agente
)


### 3.3 Las herramientas

In [0]:
from langchain.tools import tool

@tool # Los decoradores nos permiten añadirle más funcionalidades a nuestras funciones y en este caso
# le añade funcionalidades para que el agente pueda interactuar con la funcion
def extraccion_de_columnas(path: str, columna: str) -> str:
    """
    extrae las columnas dentro del dataset para su posterior análisis
    """
    df = pd.read_csv(path)[:1000]
    return list(df.columns)

@tool
def estadisticas_descriptivas(path: str, columna: str) -> str:
    """
    Calcula estadísticas descriptivas (media, mediana, desviación estándar, mínimo y máximo) 
    de una columna numérica de un CSV.
    """
    df = pd.read_csv(path)[:1000]
    if columna not in df.columns:
        return f"La columna '{columna}' no existe en el dataset."
    serie = df[columna].dropna()
    media = serie.mean()
    mediana = serie.median()
    std = serie.std()
    minimo = serie.min()
    maximo = serie.max()
    return (f"Estadísticas para '{columna}':\n"
            f"Media: {media}\n"
            f"Mediana: {mediana}\n"
            f"Desviación estándar: {std}\n"
            f"Mínimo: {minimo}\n"
            f"Máximo: {maximo}")

@tool
def conteo_valores_unicos(path: str, columna: str) -> str:
    """
    Cuenta los valores únicos y la frecuencia de cada uno en una columna del CSV.
    """
    df = pd.read_csv(path)[:1000]
    if columna not in df.columns:
        return f"La columna '{columna}' no existe en el dataset."
    conteo = df[columna].value_counts()
    return f"Conteo de valores únicos en '{columna}':\n{conteo.to_string()}"

@tool
def correlacion_columnas(path: str, columna1: str, columna2: str) -> str:
    """
    Calcula la correlación de Pearson entre dos columnas numéricas del CSV.
    """
    df = pd.read_csv(path)[:1000]
    if columna1 not in df.columns or columna2 not in df.columns:
        return f"Alguna de las columnas '{columna1}' o '{columna2}' no existe en el dataset."
    corr = df[[columna1, columna2]].corr().iloc[0,1]
    return f"La correlación de Pearson entre '{columna1}' y '{columna2}' es: {corr}"

@tool
def multiplicacion(a: int, b: int) -> int:
    """
    Multiplica dos números enteros.
    """
    return a * b

tools = [
    extraccion_de_columnas,
    estadisticas_descriptivas,
    conteo_valores_unicos,
    correlacion_columnas,
    multiplicacion
]


### 3.4 El Agente

In [0]:
from langchain.agents import create_agent

agent_executor = create_agent(llm, tools)


### 3.5 Las Instrucciones

In [0]:
user_prompt = f"""
Lee los datos del CSV en '{csv_path}',
después haz un análisis descriptivo de los datos del csv, en las columnas que tengan relevancia
y luego, de esas columnas que tienen relevancia muestrame los conteos únicos.
"""

response = agent_executor.invoke({
    "messages": [
        {
            "role": "user",
            "content": user_prompt
        }
    ]
})


### 3.6 El resultado

In [0]:
response

{'messages': [HumanMessage(content="\nLee los datos del CSV en '../data/kaggle-datasets/salaries.csv',\ndespués haz un análisis descriptivo de los datos del csv, en las columnas que tengan relevancia\ny luego, de esas columnas que tienen relevancia muestrame los conteos únicos.\n", additional_kwargs={}, response_metadata={}, id='6074c0c3-208e-4156-b894-8046d1ae5fa5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_95fef441-bd63-40bb-8847-2b3ee25e5043', 'function': {'arguments': '{"path": "../data/kaggle-datasets/salaries.csv", "columna": "salary"}', 'name': 'extraccion_de_columnas'}, 'type': 'function'}]}, response_metadata={'usage': {'prompt_tokens': 1211, 'completion_tokens': 33, 'total_tokens': 1244}, 'prompt_tokens': 1211, 'completion_tokens': 33, 'total_tokens': 1244, 'model': 'meta-llama-3.3-70b-instruct-121024', 'model_name': 'meta-llama-3.3-70b-instruct-121024', 'finish_reason': 'tool_calls'}, id='lc_run--d77cb8cf-3bcd-41e0-a881-449a96ee4747-0', tool_cal

In [0]:
for message in response["messages"]:
    if message.content:
        print(message.content)
    else:
        print(message)
    print(type(message), '\n', end = '--------------- \n')


Lee los datos del CSV en '../data/kaggle-datasets/salaries.csv',
después haz un análisis descriptivo de los datos del csv, en las columnas que tengan relevancia
y luego, de esas columnas que tienen relevancia muestrame los conteos únicos.

<class 'langchain_core.messages.human.HumanMessage'> 
--------------- 
content='' additional_kwargs={'tool_calls': [{'id': 'call_95fef441-bd63-40bb-8847-2b3ee25e5043', 'function': {'arguments': '{"path": "../data/kaggle-datasets/salaries.csv", "columna": "salary"}', 'name': 'extraccion_de_columnas'}, 'type': 'function'}]} response_metadata={'usage': {'prompt_tokens': 1211, 'completion_tokens': 33, 'total_tokens': 1244}, 'prompt_tokens': 1211, 'completion_tokens': 33, 'total_tokens': 1244, 'model': 'meta-llama-3.3-70b-instruct-121024', 'model_name': 'meta-llama-3.3-70b-instruct-121024', 'finish_reason': 'tool_calls'} id='lc_run--d77cb8cf-3bcd-41e0-a881-449a96ee4747-0' tool_calls=[{'name': 'extraccion_de_columnas', 'args': {'path': '../data/kaggle-dat

#### 3.6.1 VS python tradicional

In [0]:
def estadisticas_descriptivas(path: str, columna: str) -> str:
    """
    Calcula estadísticas descriptivas (media, mediana, desviación estándar, mínimo y máximo) 
    de una columna numérica de un CSV.
    """
    df = pd.read_csv(path)[:1000]
    if columna not in df.columns:
        return f"La columna '{columna}' no existe en el dataset."
    serie = df[columna].dropna()
    media = serie.mean()
    mediana = serie.median()
    std = serie.std()
    minimo = serie.min()
    maximo = serie.max()
    return (f"Estadísticas para '{columna}':\n"
            f"Media: {media}\n"
            f"Mediana: {mediana}\n"
            f"Desviación estándar: {std}\n"
            f"Mínimo: {minimo}\n"
            f"Máximo: {maximo}")
    
resultado_python = estadisticas_descriptivas(csv_path, 'montobruto')

print('Ejecutando nosotros la función', '\n', resultado_python, '\n')

resultado_agente = response["messages"][4].content

print('Ejecutando el agente la función', '\n', resultado_agente, '\n')


Ejecutando nosotros la función 
 Estadísticas para 'montobruto':
Media: 17633.350224948874
Mediana: 12334.14
Desviación estándar: 23253.92607222707
Mínimo: -724.1
Máximo: 409337.02 

Ejecutando el agente la función 
 Estadísticas para 'montobruto':
Media: 17633.350224948874
Mediana: 12334.14
Desviación estándar: 23253.92607222707
Mínimo: -724.1
Máximo: 409337.02 



In [0]:
%py

print(resultado_python == resultado_agente)


True


#### 3.6.2 Con una simple multpliciación

In [0]:
a = 1234565543213456765432134546765342
b = 2324354657654323456765432345465423

prompt = f"Multiplica los números {a} y {b}"

In [0]:
llm_multipliying = llm.invoke(prompt)

In [0]:
print(llm_multipliying.content)

Lo siento, pero no puedo realizar cálculos con números tan grandes de manera precisa debido a las limitaciones de mi capacidad de procesamiento y la precisión numérica. Sin embargo, puedo ofrecerte una guía sobre cómo abordar este tipo de multiplicaciones o sugerirte herramientas que podrían ayudarte.

Para multiplicar números tan grandes, generalmente se utilizan algoritmos de multiplicación de gran precisión o bibliotecas de cálculo numérico avanzadas que pueden manejar números de tamaño arbitrario. Estas bibliotecas están diseñadas para manejar operaciones con números que exceden la capacidad de los tipos de datos numéricos estándar de la mayoría de los lenguajes de programación.

Algunas opciones que podrías considerar para realizar esta multiplicación incluyen:

1. **Bibliotecas de cálculo de precisión arbitraria**: Existen bibliotecas como GMP (GNU Multiple Precision Arithmetic Library) para C y C++, o la biblioteca `decimal` en Python, que pueden manejar números de tamaño arbitr

In [0]:
agent_multiplying = agent_executor.invoke({
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ]
})

In [0]:
for message in agent_multiplying["messages"]:
    if message.content:
        print(message.content)
    else:
        print(message)
    
    print('\n', end = '--------------- \n')

Multiplica los números 1234565543213456765432134546765342 y 2324354657654323456765432345465423

--------------- 
content='' additional_kwargs={'tool_calls': [{'id': 'call_0debd090-9150-4347-9f4f-49d0ba0fcae4', 'function': {'arguments': '{ "a": 1234565543213456765432134546765342, "b": 2324354657654323456765432345465423 }', 'name': 'multiplicacion'}, 'type': 'function'}]} response_metadata={'usage': {'prompt_tokens': 1292, 'completion_tokens': 43, 'total_tokens': 1335}, 'prompt_tokens': 1292, 'completion_tokens': 43, 'total_tokens': 1335, 'model': 'meta-llama-3.3-70b-instruct-121024', 'model_name': 'meta-llama-3.3-70b-instruct-121024', 'finish_reason': 'tool_calls'} id='lc_run--96800e78-48ee-4040-a007-2cc9ab66c596-0' tool_calls=[{'name': 'multiplicacion', 'args': {'a': 1234565543213456765432134546765342, 'b': 2324354657654323456765432345465423}, 'id': 'call_0debd090-9150-4347-9f4f-49d0ba0fcae4', 'type': 'tool_call'}]

--------------- 
28695681705477381716392613270614128203340565710857802

In [0]:
a*b == int(agent_multiplying["messages"][2].content)

True


### 3.7 Casos de Uso


| Rol profesional                  | Uso típico de IA/agentes                                                         | Razón y beneficio principal                                                |
|----------------------------------|----------------------------------------------------------------------------------|---------------------------------------------------------------------------|
| CEO                              | Utilizar un LLM listo para usar y resumir informes extensos                       | Permite análisis estratégico rápido y tomar decisiones bien informadas     |
| Especialista en marketing        | Diseñar flujos automatizados en plataformas como n8n para responder chats y analizar campañas | Optimiza la interacción con clientes y mejora el rendimiento de campañas   |
| Desarrollador de software        | Integrar LangChain para procesar noticias en tiempo real y crear alertas, más agentes para trading | Automatiza tareas complejas, alimenta sistemas inteligentes y reacciona al mercado al instante |



## 4. Siguientes Pasos


Recursos para aprender mas sobre IA y agentess

ver como se puede usar en el trabajo o anivel academico

integrarlo en el dia a dia con una solucion real no solo usar por usar
